In [544]:
import pandas as pd
import sqlite3

## connection to the database

In [545]:
con = sqlite3.connect("../data/checking-logs.sqlite")

## create a new table datamart

In [546]:
datamart = pd.io.sql.read_sql(
    sql="SELECT checker.uid, labname, timestamp AS 'first_commit_ts', \
            MIN(pageviews.datetime) AS 'first_view_ts' FROM checker \
                LEFT JOIN pageviews ON checker.uid=pageviews.uid WHERE (status='ready' AND \
            numTrials=1 AND checker.uid LIKE 'user_%' AND (labname = 'laba04' OR labname = 'laba04s' OR \
                labname = 'laba05' OR labname = 'laba06' or labname = 'laba06s' OR \
                    labname = 'project1')) GROUP BY timestamp",
    con=con,
    parse_dates=['first_commit_ts', 'first_view_ts']
)
datamart.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


 ## using Pandas methods, create two dataframes: test and control

In [547]:
test = datamart.loc[datamart['first_view_ts'].notnull()]
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [548]:
control = datamart.loc[~datamart['first_view_ts'].notnull()]
control['first_view_ts'].fillna((test['first_view_ts'].mean()), inplace=True)
control.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


/var/folders/r4/bqmr8srn2017vsfr5117_g1m0000gn/T/ipykernel_4009/281808298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['first_view_ts'].fillna((test['first_view_ts'].mean()), inplace=True)


In [549]:
# (NOTE): already written to db
# test.to_sql('test', con)
# control.to_sql('control', con)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
1,1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783
2,2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783
3,5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
4,6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783
...,...,...,...,...,...
76,126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783
77,132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783
78,134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783
79,135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783


## Close connection

In [550]:
con.close()